# Orbits in space

Now that we know how to describe orbits in the perifocal frame, we need will look at how to move from this frame of reference to a more-general location in space.

This module will cover:
- Geocentric equatorial frame
- Orbital elements
- Gibbs problem
- Gauss problem/Lambert theorem

In [2]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

## Geocentric equatorial frame

In the geocentric equatorial frame, the Earth lies at the center and spins,
where the $X-Y$ plane aligns with the equatorial plane of the planet and
the planet's axis of rotation points in the $Z$ direction of the frame.

Note that this frame does not align with the plane of the ecliptic (the solar plane),
since the Earth's axis is tilted 23.44°.

:::{figure,myclass} fig-gef
<img src="../images/geocentric_equatorial.png" alt="Depiction of geocentric equatorial frame" class="bg-white mb-1" width="300px">

Diagram of geocentric equatorial frame. 
Original source by [Tfr000](https://en.wikipedia.org/wiki/File:Ra_and_dec_rectangular.png),
shared under [CC BY-SA](https://creativecommons.org/licenses/by-sa/3.0/deed.en)
:::

The $X$ axis of the geocentric equatorial frame always points in the same direction,
towards the vernal equinox. This direction is also referred to as the first point
in Aries, or with &#x2648;, since this pointed to the constellation Aries back
when the Greeks came up with the idea. However, due to the precession of the Earth's tilt,
the direction changes over time, and it now points towards Pisces. In the year 2600,
the vernal equinox will point towards Aquarius.

Due to this precession, for precise orbital determinations we need to specify the
direction of $X$ via the appropriate 50-year **epoch**. We are currently in the 
2000 epoch, but in 2025 we will pass into the 2050 epoch.

For an object in space, we can specify its position in the geocentric equatorial
frame via its distance from the center ($r$), the angle away from the $X$ direction 
in the equatorial plane is the right ascension ($\alpha$), and the angle above
the plane is the declination ($\delta$).

## Orbital elements

In the perifocal frame, we can specify an object in orbit using the eccentricity $e$,
the specific angular momentum $h$, and the true anomaly $\theta$.
To specify the orientation of this plane in the geocentric equatorial frame,
we need three additional quantities.

:::{figure,myclass} fig-orbital-elements
<img src="../images/orbital_elements.*" alt="Diagram with classical orbital elements" class="bg-white mb-1" width="400px">

Diagram showing the orbital elements. 
Note that this uses $\nu$ for true anomaly, while we usually use $\theta$ here.
Source: [Lasunncty](https://commons.wikimedia.org/wiki/File:Orbit1.svg) at the English Wikipedia / [CC BY-SA](http://creativecommons.org/licenses/by-sa/3.0/)
:::

The additional three quantities needed are the:
* right ascension $\Omega$, which is the angle between the reference direction $\vec{X}$, or &#x2648;, and the direction of the line of the ascending node $\vec{N}$;
* inclination $i$, the angle of the perifocal frame away from the reference plane; and
* argument of periapse/perigee $\omega$, the angle between the line of ascending node and direction of periapse $\vec{e}$.

The orbit is prograde if $i <$ 90°, and retrograde if $i >$ 90°.

Now, we have the six **orbital elements** needed to uniquely specify an orbit in a two-body system: $h, e, \theta, \Omega, i, \omega$.